---
title: "Antenatal Care Visits Analysis"
execute:
  echo: true
format:
  html:
    code-fold: true
    toc: true
    toc-location: left
    anchor-sections: true
    jupyter: python3
---

In [1]:
import pandas as pd

# Load the files
indicator_df = pd.read_csv('unicef_indicator_1.csv')
metadata_df = pd.read_csv('unicef_metadata.csv')


# Introduction
*Understanding antenatal care coverage is crucial for improving maternal and child health worldwide.  
This report uses UNICEF global datasets to visualize antenatal care trends, economic correlations, and regional disparities.  
All visualizations are created using Python, Plotly, and Quarto.*

# **1. Global Antenatal Care Visits Map**

*Figure: Global view highlighting disparities in antenatal care visits, with smooth interactive hover to explore country-specific insights.*


In [2]:
import plotly.express as px

# World Map: Global Antenatal Care Visits
fig = px.choropleth(
    indicator_df,
    locations="country",
    locationmode="country names",
    color="obs_value",
    hover_name="country",
    hover_data={"country": False, "obs_value": True},
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Global Antenatal Care Visits Percentage via World Map",
    labels={"obs_value": "Antenatal Care Visits (%)"}
)

fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br><br>" +
                  "Antenatal Care Visits: %{z:.1f}%<extra></extra>"
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type="natural earth",
        landcolor="lightgray",
        showcountries=True,
        countrycolor="white"
    ),
    coloraxis_colorbar_title="Antenatal Care Visits (%)",
    hoverlabel=dict(
        bgcolor="white",
        font_size=13,
        font_family="Arial"
    ),
    margin=dict(l=0, r=0, t=50, b=0),
    transition_duration=700
)

fig.show()

*Global view highlighting disparities in antenatal care visits, with smoother hover insights for a user-friendly exploration.*

---

> ***The global choropleth map reveals stark regional inequalities in antenatal care visits. Sub-Saharan Africa and parts of South Asia show significantly lower percentages compared to Europe, North America, and Australia. The visualization emphasizes the urgent need for healthcare interventions in underserved regions, especially across Africa and parts of Asia.***



## **2. Representative Sample of Countries for Antenatal Care Visits (Bar Chart)**


*Figure: A representative sample across low to high-performing countries, offering a spectrum of antenatal care coverage through an interactive bar visualization.*


In [3]:
import pandas as pd
import plotly.express as px

# Bar Chart: Representative Sample of Countries for Antenatal Care Visits
antenatal_data_binned = indicator_df.copy()
antenatal_data_binned['bin'] = pd.qcut(antenatal_data_binned['obs_value'], q=5, labels=False)

subset_list = [
    antenatal_data_binned[antenatal_data_binned['bin'] == bin_num]
    .sample(min(3, len(antenatal_data_binned[antenatal_data_binned['bin'] == bin_num])), random_state=42)
    for bin_num in antenatal_data_binned['bin'].unique()
]

bar_data = pd.concat(subset_list)
bar_data = bar_data.groupby('country', as_index=False)['obs_value'].mean()
bar_data = bar_data.sort_values('obs_value')

fig = px.bar(
    bar_data,
    x="country",
    y="obs_value",
    color="obs_value",
    text="obs_value",
    title="Representative Sample of Countries for Antenatal Care Visits",
    labels={"obs_value": "Antenatal Care Visits (%)"},
    color_continuous_scale=px.colors.sequential.Plasma
)

fig.update_traces(
    texttemplate='%{text:.0f}%',
    textposition='outside',
    hovertemplate="<b>Country:</b> %{x}<br><b>Antenatal Care Visits:</b> %{y:.2f}%<extra></extra>",
    marker=dict(opacity=0.85, line=dict(width=1.5, color='DarkSlateGrey')),
    selector=dict(type='bar')
)

fig.update_layout(
    xaxis_title="Country",
    yaxis_title="Antenatal Care Visits (%)",
    coloraxis_colorbar_title="Antenatal Care (%)",
    hovermode="closest",
    hoverlabel=dict(bgcolor="white", font_size=13, font_family="Arial"),
    plot_bgcolor="rgba(0,0,0,0)",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    transition_duration=500
)

fig.show()

*A representative sample across low to high-performing countries, offering a spectrum of antenatal care coverage through an interactive bar visualization.*


---

> By sampling countries across different performance levels, we observe a progressive trend where antenatal care visits vary from critically low (below 30%) to near-universal coverage (above 90%). This highlights how some nations have successfully scaled maternal healthcare programs, while others continue to face accessibility and systemic challenges. Even within similar economic groups, antenatal care rates vary significantly, suggesting that national healthcare policies and outreach efforts play a critical role.


## 3. GDP per Capita vs Antenatal Care Visits (Scatter Plot)

Figure: Interactive scatter plot illustrating the correlation between GDP per capita and antenatal care visits, enhanced with a regression trendline for clear pattern visibility.


In [4]:
# Prepare Antenatal Care Data from indicator_df
antenatal_data = indicator_df[indicator_df['indicator'].str.contains('antenatal', case=False, na=False)]
antenatal_data = antenatal_data.rename(columns={'time_period': 'year'})

# Prepare Life Expectancy Data from metadata_df
# Pick only needed columns
life_expectancy_data = metadata_df[['country', 'year', 'Life expectancy at birth, total (years)']]
life_expectancy_data = life_expectancy_data.rename(columns={'Life expectancy at birth, total (years)': 'Life Expectancy (Years)'})

# 3. Merge Both DataFrames
merged_data = pd.merge(
    antenatal_data[['country', 'year', 'obs_value']],
    life_expectancy_data,
    on=['country', 'year']
)

# 1. Take the latest year per country
merged_data_latest = merged_data.sort_values('year').groupby('country').tail(1)

#Rename the columns in merged_data_latest
merged_data_latest = merged_data_latest.rename(columns={'obs_value': 'obs_value_Antenatal', 'Life Expectancy (Years)': 'obs_value_GDP'})

# 1. Create colorful and interactive scatter plot
fig = px.scatter(
    merged_data_latest,
    x='obs_value_GDP',
    y='obs_value_Antenatal',
    trendline="ols",
    hover_name='country',
    title="Interactive Scatter Plot: GDP per Capita vs Antenatal Care Visits (2023)",
    labels={
        'obs_value_GDP': 'GDP per Capita (Constant 2015 US$)',
        'obs_value_Antenatal': 'Antenatal Care Visits (%)'
    },
    opacity=0.8,
    size_max=15
)

fig.update_traces(
    marker=dict(size=10, line=dict(width=0.5, color='DarkSlateGrey'))
)

fig.update_layout(
    height=650,
    width=1000,
    xaxis_title="GDP per Capita (Constant 2015 US$ in %)",
    yaxis_title="Antenatal Care Visits (%)",
    hovermode="closest",
    showlegend=True,
    legend_title_text='Legend'
)

from plotnine import ggplot, aes, geom_point, geom_smooth, theme_minimal, labs

(
    ggplot(merged_data_latest, aes(x='obs_value_GDP', y='obs_value_Antenatal')) +
    geom_point(color="blue") +
    geom_smooth(method='lm', color="red", se=True) +
    theme_minimal() +
    labs(
        title="Scatter Plot with Linear Regression: GDP vs Antenatal Care",
        x="GDP per Capita (Constant 2015 US$ in %)",
        y="Antenatal Care Visits (%)"
    )
)

fig.show()

*Interactive scatter plot illustrating the correlation between GDP and antenatal care visits, enhanced with a regression line for easy trend recognition.*


---


> ***The scatter plot shows a positive correlation between a country’s GDP per capita and its antenatal care coverage. Higher-income countries tend to have higher antenatal care rates. However, some outliers exist, suggesting that economic wealth alone doesn’t guarantee healthcare access — policy priorities, healthcare infrastructure, and education also play a critical role. However, several outliers reveal that economic prosperity does not automatically translate to improved maternal health services without targeted investment.***


## **4. Antenatal Care Visits vs Life Expectancy (Time Series)**

Figure: Dual time-series exploration of antenatal care visits and life expectancy, emphasizing the pivotal 2010 milestone with interactive hover insights.


In [5]:
import pandas as pd
import plotly.express as px

# Prepare Antenatal Care Data
antenatal_data = indicator_df[indicator_df['indicator'].str.contains('antenatal', case=False, na=False)]
antenatal_data = antenatal_data.rename(columns={'time_period': 'year'})

# Prepare Life Expectancy Data
life_expectancy_data = metadata_df[['country', 'year', 'Life expectancy at birth, total (years)']]
life_expectancy_data = life_expectancy_data.rename(columns={'Life expectancy at birth, total (years)': 'Life Expectancy (Years)'})

# Merge Data
merged_data = pd.merge(
    antenatal_data[['country', 'year', 'obs_value']],
    life_expectancy_data,
    on=['country', 'year']
)

# Melt for Plotly
plot_data_melted = merged_data.melt(
    id_vars=["country", "year"],
    value_vars=["obs_value", "Life Expectancy (Years)"],
    var_name="Indicator Type",
    value_name="Value"
)

# Rename indicators
plot_data_melted['Indicator Type'] = plot_data_melted['Indicator Type'].replace({
    'obs_value': 'Antenatal Care Visits (%)'
})

# Select Countries
selected_countries = ['Germany', 'United States', 'India', 'Ethiopia', 'Bangladesh']
plot_data_melted = plot_data_melted[plot_data_melted['country'].isin(selected_countries)]

# Round all the Measurement values to 2 decimal places
plot_data_melted["Value"] = plot_data_melted["Value"].round(2)

fig = px.line(
    plot_data_melted,
    x="year",
    y="Value",
    color="country",
    line_dash="Indicator Type",
    markers=True,
    line_shape="spline",
    custom_data=["country", "Indicator Type"],
    hover_name="country",
    hover_data={"country": False, "year": True, "Indicator Type": True, "Value": True},
    # Remove hovertemplate
    title="Interactive Trend Analysis: Antenatal Care Visits vs Life Expectancy Highlighting a Pivotal Milestone in 2010."
)

# Update layout
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Measurement (Percentage for Antenatal Care / Years for Life Expectancy)",
    height=650,
    width=950,
    legend_title="Country",
    hovermode="x unified"
)

fig.add_vline(
    x=2010,
    line_width=2,
    line_dash="dash",
    line_color="black",
    annotation_text="2010 Milestone",
    annotation_position="top",
    annotation_font_size=12
)

# Add hovertemplate to each trace after creating the figure
for trace in fig.data:
    trace.hovertemplate = (
        "<b>%{customdata[0]}</b><br>" +  # Country
        "Year: %{x}<br>" +               # Year
        "Value: %{y}<br>" +              # Value
        "%{customdata[1]}"               # Indicator (either Antenatal or Life Expectancy)
    )
fig.show()

*Dual time-series exploration of antenatal care and life expectancy, emphasizing the pivotal 2010 milestone with interactive hover details across countries.*

---



> ***Both antenatal care visits and life expectancy show an upward trend across most countries analyzed. A pivotal surge around 2010 highlights global initiatives like the Millennium Development Goals (MDGs) that emphasized maternal health. This reinforces how international policy frameworks can have a lasting impact on public health outcomes. The synchronised rise in antenatal care visits and life expectancy after 2010 reflects global efforts towards Millennium Development Goals (MDGs) and healthcare improvements.***



## **5. Regional Distribution of Global Antenatal Care Visits (Pie Chart)**

Figure: Pie chart presenting the regional proportions of antenatal care coverage, highlighting both the volume and the disparities.


In [6]:
import plotly.express as px

# Pie Chart: Regional Distribution of Global Antenatal Care Visits
fig = px.pie(
    indicator_df,
    names='country',
    values='obs_value',
    title="Regional Distribution of Global Antenatal Care Visits",
    hole=0.4
)

fig.update_traces(
    pull=[0.08] * len(indicator_df),
    marker=dict(line=dict(color='black', width=2)),
    hoverinfo='label+percent+value',
    textinfo='percent+label',
    textposition='inside',
    insidetextorientation='radial'
)

fig.update_layout(
    height=600,
    width=800,
    legend_title="Region CLICK ME!",
    hoverlabel=dict(bgcolor="white", font_size=13, font_family="Arial"),
    uniformtext_minsize=12,
    uniformtext_mode='hide'
)

fig.show()

*Pie chart showing regional proportions of antenatal care, with hover-enabled clarity for detailed regional comparisons.*

---



> ***The pie chart showcases how antenatal care is unevenly distributed across world regions. Regions like Europe and North America have significantly higher shares, while Sub-Saharan Africa still holds a disproportionately low percentage, further underlining systemic global health inequalities. High population regions contribute heavily to the global total of antenatal visits, but the pie size can mask the fact that many individual countries still face gaps in service quality.***





***Conclusion***


*   Significant regional disparities exist in antenatal care coverage, with lower rates concentrated in developing regions.
*   Economic prosperity strongly correlates with better access to antenatal care, as shown by the relationship between GDP per capita and antenatal care visits.

*   A pivotal improvement around 2010 suggests that global health initiatives like the Millennium Development Goals had a real positive impact.

*   Healthcare policies and infrastructure, not just economic wealth, play a crucial role in expanding maternal care accessibility.

*   Targeted regional interventions and sustained investment are essential to close existing gaps and improve maternal and child health worldwide.




